# 7. Dimensionalitätsreduktion - Modeling

## Anforderungen an Projektumsetzung: Dimensionalitätsreduktion

---
**AUFGABE:**

- Definieren Sie für Ihren Datensatz ein oder mehrere Ziele, die Sie mit Hilfe von Dimensionsreduktion der Daten erreichen wollen.
- Führen Sie mit dem Algorithmus Ihrer Wahl eine Dimensionsreduktion auf Ihren Daten durch.
- Setzen Sie ggf. die Parameter des Algorithmus zur Dimensionsreduktion mit Hilfe einer Pipeline.
- Beschreiben Sie Ihre Ergebnisse. Haben Sie Ihr(e) Ziel(e) erreicht?

----
**Ziele**
1. Verstehen, was die wichtigen Features in unserem Datensatz sind und eine intuitivere Darstellung der Daten ermöglichen. 
2. Aufgrund der vielen Datensätze entstehen hohe Rechenkapazitäten. Daher sollen die Trainings- und Testzeiten verkürzt werden, unter der Verwendung von weniger (aber gleichermaßen aussagekräftige) Features.

**Diese Ziele sollen mit Hilfe von dem PCA-Algorithmus erreicht werden. D.h. wir wollen die Dimensionen/Features reduzieren, ohne die Varianz damit zu veschlechtern - eher verbessern.**
-  je höher die Varianz desto aussagekräftiger das Feature
- PCA soll Aufschluss darüber geben, wie die Varianz der Daten verteilt ist und welche Features gemeinsam variieren
- zu Untersuchen ist also, welche Features aussagekräftig zur/über Zielvariable ist --> können wir Features finden, die viel über die Kategorien aussagen (Rating, Max. Installs, Price?)

In [1]:
# Imports
from sklearn import datasets, svm, metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans

import pandas as pd

In [2]:
apps = pd.read_csv("Daten/Google-Playstore_Edit2.csv")

In [3]:
# alle Kategorien löschen, die für uns als Unternehmen irrelevant und für eine Entwicklung ausgeschlossen sind
less_apps = apps[apps['Category'] == 'Action'] + apps[apps['Category'] == 'Arcade'] + apps[apps['Category'] == 'Beauty'] + apps[apps['Category'] == 'Casino'] + apps[apps['Category'] == 'Comics'] + apps[apps['Category'] == 'Dating'] + apps[apps['Category'] == 'Educational'] + apps[apps['Category'] == 'Puzzle'] + apps[apps['Category'] == 'Racing'] + apps[apps['Category'] == 'Role Playing'] + apps[apps['Category'] == 'Shopping'] + apps[apps['Category'] == 'Trivia'] + apps[apps['Category'] == 'Video Players & Editors'] 
            
apps.drop(less_apps.index, axis=0, inplace=True)            

In [4]:
apps.groupby('Category')['Category'].count()

Category
Adventure             23196
Art & Design          18538
Auto & Vehicles       18278
Board                 10588
Books & Reference    116726
Business             143761
Card                   8179
Casual                50797
Communication         48159
Education            241075
Entertainment        138268
Events                12839
Finance               65456
Food & Drink          73920
Health & Fitness      83501
House & Home          14369
Libraries & Demo       5196
Lifestyle            118324
Maps & Navigation     26722
Medical               32063
Music                  4202
Music & Audio        154898
News & Magazines      42804
Parenting              3810
Personalization       89210
Photography           35552
Productivity          79686
Simulation            23276
Social                44729
Sports                47478
Strategy               8525
Tools                143976
Travel & Local        67282
Weather                7245
Word                   8630
Name: Categ

In [5]:
# Datensatz random auf die Hälfte reduzieren und neues DataFrame erstellen
half_apps = apps.sample(frac = 0.5)

In [6]:
# Alle Spalten mit Unique-Werten werden gedropped - zu viel Rechenkapa notwendig
half_apps.drop(columns=['App Name', 'App Id', 'Developer Id', 'Developer Website','Minimum Android', 'Developer Email', 'Privacy Policy', 'Released', 'Scraped Time', 'Last Updated'], inplace=True)

In [7]:
# Umwanldung in Float-Werte
half_apps['Free']             = half_apps['Free'].astype(float)
half_apps['Ad Supported']     = half_apps['Ad Supported'].astype(float)
half_apps['Editors Choice']   = half_apps['Editors Choice'].astype(float)
half_apps['In App Purchases'] = half_apps['In App Purchases'].astype(float)
half_apps['Maximum Installs'] = half_apps['Maximum Installs'].astype(float)

In [8]:
# Selektion von den Spalten vom Typ object
half_apps = half_apps.dropna()
half_apps.select_dtypes(include=['object'])

,Category,Installs,Currency,Content Rating
354382,Casual,"100,000+",USD,Everyone
804770,Sports,"10,000+",USD,Everyone
1463066,Finance,10+,USD,Everyone
1539508,Health & Fitness,100+,USD,Everyone
1140381,Health & Fitness,"1,000+",USD,Mature 17+
...,...,...,...,...
1691810,Food & Drink,100+,USD,Everyone
1197661,Education,50+,USD,Everyone
472029,Education,"5,000+",USD,Everyone
1088615,Productivity,"100,000+",USD,Everyone


In [9]:
# Aufteilung in Listen mit numerischen und mit noch kategorischen Werten
numerical_cols = list(half_apps.select_dtypes(include="float").columns)
categorical_cols = list(half_apps.select_dtypes(include="object").columns)

In [10]:
# Löschen von Category, da dies dann als Zielklasse verwendet werden soll
categorical_cols.remove("Category")
categorical_cols

['Installs', 'Currency', 'Content Rating']

In [11]:
# Da Klassifikation nur mit numerischen Daten funktioniert, werden mittels
# One-Hot-Endcoding aus den kategorischen Spalten, numerische Daten generiert
X_dumm = pd.get_dummies(half_apps[categorical_cols])

In [12]:
# Zusammenfügen beider numerischen Listen
X = pd.concat([half_apps[numerical_cols], X_dumm], axis = 1)

In [13]:
X.describe()

,Rating,Rating Count,Minimum Installs,Maximum Installs,Free,Price,Size,Ad Supported,In App Purchases,Editors Choice,...,Currency_SGD,Currency_USD,Currency_VND,Currency_XXX,Content Rating_Adults only 18+,Content Rating_Everyone,Content Rating_Everyone 10+,Content Rating_Mature 17+,Content Rating_Teen,Content Rating_Unrated
count,974906.000000,9.749060e+05,9.749060e+05,9.749060e+05,974906.000000,974906.000000,974906.000000,974906.000000,974906.00000,974906.000000,...,974906.000000,974906.000000,974906.000000,974906.000000,974906.000000,974906.000000,974906.000000,974906.000000,974906.000000,974906.000000
mean,2.174457,2.228971e+03,1.532630e+05,2.761469e+05,0.979799,0.109705,17.824605,0.492032,0.07231,0.000265,...,0.000001,0.999492,0.000001,0.000500,0.000062,0.880553,0.012081,0.023635,0.083604,0.000066
std,2.108790,1.688526e+05,1.454329e+07,2.285354e+07,0.140687,2.652238,23.320178,0.499937,0.25900,0.016266,...,0.001013,0.022527,0.001013,0.022345,0.007845,0.324314,0.109248,0.151910,0.276793,0.008102
min,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000e+00,5.000000e+01,8.800000e+01,1.000000,0.000000,4.500000,0.000000,0.00000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,2.800000,6.000000e+00,5.000000e+02,7.080000e+02,1.000000,0.000000,9.200000,0.000000,0.00000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,4.300000,4.000000e+01,5.000000e+03,7.115750e+03,1.000000,0.000000,22.000000,1.000000,0.00000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,5.000000,1.202062e+08,1.000000e+10,1.205763e+10,1.000000,399.990000,996.000000,1.000000,1.00000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
y = half_apps['Category'] #.copy()
print(f"X und y haben gleiche Anzahl: {X.shape[0] == y.shape[0]}")

X und y haben gleiche Anzahl: True


In [15]:
label_encoder = LabelEncoder()

In [16]:
y = label_encoder.fit_transform(y) # macht alles zu 0, 1, 2,3 ...

In [17]:
# Daten in Trainings- und Test aufteilen
X_train, X_test1, y_train, y_test1 = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Testdaten in Entwicklung und echten Test aufteilen (50-50, stratifiziert)
X_dev, X_test, y_dev, y_test = train_test_split(X_test1, y_test1, test_size=0.5, stratify=y_test1, random_state=42)

In [18]:
# Klassifizierer vorbereiten: Support Vector Machine 
svm_classifier = svm.SVC()

# Auf den Trainingsdaten lernen
svm_classifier.fit(X_train[:5000], y_train[:5000])

# Vorhersagen für die Testdaten machen und berichten
predicted = svm_classifier.predict(X_dev)

print(f"Classification report for classifier {svm_classifier}:\n"
      f"{metrics.classification_report(y_dev, predicted)}\n")

Classification report for classifier SVC():
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1129
           1       0.00      0.00      0.00       913
           2       0.00      0.00      0.00       845
           3       0.00      0.00      0.00       512
           4       0.00      0.00      0.00      5792
           5       0.00      0.00      0.00      6806
           6       0.00      0.00      0.00       400
           7       0.00      0.00      0.00      2460
           8       0.00      0.00      0.00      2319
           9       0.12      0.99      0.22     11806
          10       0.13      0.00      0.00      6747
          11       0.00      0.00      0.00       626
          12       0.00      0.00      0.00      3092
          13       0.00      0.00      0.00      3555
          14       0.00      0.00      0.00      4030
          15       0.00      0.00      0.00       681
          16       0.00      0.00    

/home/user/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/user/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/user/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
# Klassifizierer vorbereiten: Gaussian Naive Bayes 
g_classifier = GaussianNB()

# Auf den Trainingsdaten lernen
g_classifier.fit(X_train[:5000], y_train[:5000])

# Vorhersagen für die Testdaten machen und berichten
predicted = g_classifier.predict(X_dev)

print(f"Classification report for classifier {g_classifier}:\n"
      f"{metrics.classification_report(y_dev, predicted)}\n")

Classification report for classifier GaussianNB():
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1120
           1       0.00      0.00      0.00       921
           2       0.00      0.00      0.00       858
           3       0.00      0.00      0.00       515
           4       0.07      0.06      0.07      5781
           5       0.08      0.97      0.14      6806
           6       0.00      0.00      0.00       388
           7       0.00      0.00      0.00      2444
           8       0.00      0.00      0.00      2300
           9       0.06      0.00      0.00     11857
          10       0.07      0.00      0.01      6746
          11       0.00      0.00      0.00       626
          12       0.10      0.00      0.00      3094
          13       0.00      0.00      0.00      3538
          14       0.00      0.00      0.00      4065
          15       0.00      0.00      0.00       685
          16       0.00      0

/home/user/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/user/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/user/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


- GaussianNB noch schlechter als SVM bei unseren Daten, daher eher SVM für die Pipeline

***Feature-Auswahl***

In [ ]:
# Wie viel Varianz muss mindestens vorhanden sein?
dim_reduction = VarianceThreshold(0.1)

# Klassifizierer wählen
classifier = svm.SVC()

# Pipeline erstellen

pipeline = Pipeline([('dim_reduction', dim_reduction),
                     ('classifier', classifier)])

pipeline.fit(X_train, y_train)

# Varianz der einzelnen Features ausgeben
# Hier können Sie sehen, welche Features bei Schwellenwert 0.1 
# ausgefiltert werden.
print(dim_reduction.variances_)

- ohne Ergebnis, weil zu lang 
----
***Pricipal Components Analysis - PCA***

In [23]:
# PCA initialisieren; die Lösung soll mindestens 90% der Varianz in den Daten bewahren
dim_reduction = PCA(n_components=0.90)

# Klassifizierer wählen
svm_classifier = svm.SVC()

# Pipeline erstellen
pipeline = Pipeline([('dim_reduction', dim_reduction), ('classifier', svm_classifier)])

pipeline.fit(X_train[:5000], y_train[:5000])
# 2000 --> [0.98226249]
# 500 --> [0.97185566]

# Ausgeben, wie viel Varianz die einzelnen Hauptkomponenten erklären.
print(dim_reduction.explained_variance_ratio_)

[0.98330995]


----
- aus 46 Feautures haben wir eine Hauptkomponente mit einer Varianz von 0.98, die am aussagekräftigsten ist und natürlich unbekannt ist
- d.h. wir sparen 45 (46-1) Dimensionen ein und mit bereits einer Hauptkomponente wird die gewünschte Mindestvarianz erreicht
    - Die Dimensionalität eines Datensatzes ist durch die Anzahl der vorliegenden Features gegeben.
----

In [24]:
# Vorhersagen für die Testdaten nach einer Dimensionsreduktion machen und berichten
predicted = pipeline.predict(X_dev)

print(f"Classification report for VarianceThreshold dimensionality reduction:\n"
      f"{metrics.classification_report(y_dev, predicted)}\n")

Classification report for VarianceThreshold dimensionality reduction:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1129
           1       0.00      0.00      0.00       913
           2       0.00      0.00      0.00       845
           3       0.00      0.00      0.00       512
           4       0.00      0.00      0.00      5792
           5       0.00      0.00      0.00      6806
           6       0.00      0.00      0.00       400
           7       0.00      0.00      0.00      2460
           8       0.00      0.00      0.00      2319
           9       0.12      0.99      0.22     11806
          10       0.12      0.00      0.00      6747
          11       0.00      0.00      0.00       626
          12       0.00      0.00      0.00      3092
          13       0.00      0.00      0.00      3555
          14       0.00      0.00      0.00      4030
          15       0.00      0.00      0.00       681
          1

/home/user/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/user/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/user/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


--------------------
***Clustering***

In [ ]:
# KMeans initialisieren: So viele Cluster wie Klassen
dim_reduction = KMeans(n_clusters=35)

# Klassifizierer wählen
classifier = svm.SVC()

# Pipeline erstellen; KMeans als Transformer führt die Cluster-basierte Dimensionsreduktion durch

pipeline = Pipeline([('dim_reduction', dim_reduction),
                     ('classifier', classifier)])

pipeline.fit(X_train, y_train)

- erneut ohne Ergebnisse, weil zu rechenintensiv (Berechnung der Abstände jedes Datenpunktes zu den 35 Cluster-Zentroiden)
----
**AUFGABE:**
- Beschreiben Sie Ihre Ergebnisse. Haben Sie Ihr(e) Ziel(e) erreicht?

**Recap Ziele**
1. Verstehen, was die wichtigen Features in unserem Datensatz sind und eine intuitivere Darstellung der Daten ermöglichen. 
2. Aufgrund der vielen Datensätze entstehen hohe Rechenkapazitäten. Daher sollen die Trainings- und Testzeiten verkürzt werden, unter der Verwendung von weniger (aber gleichermaßen aussagekräftige) Features.
-----
1. Nein, da es bei eine Hauptkomponente keinen wirklichen Aufschluss darüber gibt, wie die Varianz der Daten verteilt ist und welche Features gemeinsam variieren. Des Weiteren werden bei einer Datentransformation mit PCA nicht mehr unsere Ursprungsfeatures verwendet, sodass wir nicht die Eigenschaft des Feautres bzw. Hauptkomponente kennen/wissen. 
2. Durch die Dimensionalitätsreduktion wurde tatsächlich die Trainings- und Testzeiten verkürzt im Vergleich zur 'normalen/einfachen' Klassifikation (Feature-Selektion und SVC)
--> Dennoch fällt die Vorhersage auf diese Testdaten schlechter aus, als auf die Vorhersage auf der Testdaten bei der einfachen Klassifikation